In [3]:
import spacy
from collections import Counter
from nltk.tokenize import sent_tokenize
from spacy import displacy

nlp = spacy.load("en_core_web_sm")


In [163]:
def get_adjective_phrases(text) -> list:
    
    def get_to_object(token) -> int:
        print(token)
        for child in token.children:
            subtoken = list(child.children)
            subtoken = [token for token in subtoken if token.dep_ == 'prep']
            if (child.pos_ in ['NOUN', 'PRON', 'PROPN']) and (child.dep_ == 'pobj'):
                print("Child", child)
                return child
            elif (len(subtoken) > 0):
                test_arr = [get_to_object(text) for text in subtoken if get_to_object(text) is not None]
                return test_arr[0]
    
    doc = nlp(text)
    phrases = []
    for token in doc:
        phrase = ''
        if (token.pos_ == 'ADJ') and (token.dep_ in ['ROOT', 'acomp', 'amod', 'conj']):
            phrase += token.text
            adjective_position = token.i
            for subtoken in token.children:
				# first rule: if there is an adverb that modifies the adjective
				# we add it to the phrase in front of the adjective
                if (subtoken.pos_ == 'ADV') and (subtoken.dep_ == 'advmod'):
                    phrase = subtoken.text + ' ' + phrase
				# second rule: if there is a preposition - indicating that
				# there is an object that gives us more info about the prep
				# we add all tokens up until the noun
                if (subtoken.pos_ in ['ADP', 'SCONJ']) and (subtoken.dep_ == 'prep'):
                    try:
                        noun_position = get_to_object(subtoken)
                        print("here", noun_position)
                        for i in range(adjective_position + 1, noun_position + 1):
                            phrase += ' ' + doc[i].text
                    except TypeError:
                        pass
        elif (token.pos_ == 'ADV') and (token.dep_ == 'advmod'):
            phrase += token.text
            adverb_position = token.i
            for subtoken in token.children:
				# first rule: if there is an adverb that modifies the adjective
				# we add it to the phrase in front of the adjective
                if (subtoken.pos_ == 'ADJ') and (subtoken.dep_ in ['ROOT', 'acomp', 'amod', 'advmod']):
                    phrase = subtoken.text + ' ' + phrase
				# second rule: if there is a preposition - indicating that
				# there is an object that gives us more info about the prep
				# we add all tokens up until the noun
		# need to fix so that it does not get random adverbs and adjectives
        elif (token.pos_ == 'AUX') and (token.dep_ == 'ROOT'):
            adv = adj =  None
            for subtoken in token.children:
                if subtoken.dep_ == 'advmod':
                    if adv == None:
                        adv = subtoken
                if subtoken.dep_ == 'acomp':
                    if adj == None:
                        adj = subtoken
                if adv != None and adj != None:
                    if adv.i +1 == adj.i:
                        phrase += adv.text + " " + adj.text + " "
		# since it is a phrase, it needs to have more than one word
		# i.e. a lone adjective does not constitute an adjective phrase
        if len(phrase.split()) > 1:
            phrases.append(phrase)
    return phrases

In [164]:
get_adjective_phrases("I am tired of dealing with all of the mess that has been created by those people")

of
with
of
Child mess
of
Child mess
with
of
Child mess
of
Child mess
here mess


[]

In [104]:
doc = nlp("I am tired of dealing with all of the mess that has been created by those people")
displacy.render(doc, style="dep")

In [76]:
for token in doc:
    print(token.text, token.dep_)
    for child in token.children:
        print("Token", token.text, token.pos_, token.dep_, "Child", child.text, child.dep_)

The det
beauty nsubj
Token beauty NOUN nsubj Child The det
Token beauty NOUN nsubj Child of prep
of prep
Token of ADP prep Child song pobj
this det
song pobj
Token song NOUN pobj Child this det
is ROOT
Token is AUX ROOT Child beauty nsubj
Token is AUX ROOT Child amazing acomp
amazing acomp
Token amazing ADJ acomp Child and cc
Token amazing ADJ acomp Child clear conj
and cc
marvelously advmod
clear conj
Token clear ADJ conj Child marvelously advmod
